In [3]:
# Cell 1 – imports & paths
import numpy as np, pandas as pd, os
VEC_PATH  = "/home/ivan/PycharmProjects/MPr/notebooks/embeddings/audio_vectors.npy"
FILE_PATH = "/home/ivan/PycharmProjects/MPr/notebooks/embeddings/audio_filenames.npy"
META_CSV  = "/home/ivan/PycharmProjects/MPr/csv_files/spotify_data.csv"        # path to uploaded csv

# Cell 2 – load vectors & filenames
vectors   = np.load(VEC_PATH)
files     = np.load(FILE_PATH)
N         = len(files)

# Cell 3 – read metadata & build map filename → (artist, genre)
meta = pd.read_csv(META_CSV)
id2row = {row.track_id: row for _, row in meta.iterrows()}

def id_from_fname(p):
    return os.path.basename(p).split('.')[0].lstrip("0")   # '066689.mp3' -> '66689'

artist_groups = {}
genre_groups  = {}

for idx, f in enumerate(files):
    tid = id_from_fname(f)
    if tid not in id2row: continue
    artist = id2row[tid].artist_name
    genre  = id2row[tid].genre
    artist_groups.setdefault(artist, []).append(idx)
    genre_groups.setdefault(genre, []).append(idx)

# Cell 4 – build co-occurrence counts
co = np.zeros((N, N), dtype='float32')

def add_playlist(idxs):
    for i in idxs:
        for j in idxs:
            if i != j:
                co[i, j] += 1

for pl in artist_groups.values(): add_playlist(pl)
for pl in genre_groups.values():  add_playlist(pl)

# Cell 5 – normalize rows 0-1 and save
co /= np.maximum(co.max(axis=1, keepdims=True), 1)
np.save("/home/ivan/PycharmProjects/MPr/notebooks/embeddings/co_matrix.npy", co)
print("Saved co_matrix with shape", co.shape)


Saved co_matrix with shape (300, 300)
